In [1]:
source("../../../wwa/methods_extRemes.r")

In [8]:
gmst <- read.csv("../../../wwa/90_data/gmst.csv", row.names = "year")
gmst <- gmst - gmst["2022","gmst"]+1.2
range(gmst[sapply(1950, toString),])

[1] 0.1635417 0.1635417

### **Max likelihood estimates at each grid cell**

#### **ML fit at each grid cell**

In [98]:
gmst <- read.csv("../../../wwa/90_data/gmst.csv", row.names = "year")

times <- sapply(list.files("data"), function(fnm) {

    era5 <- read.csv(paste0("data/", fnm), skip = 3, header = F, row.names = "V1")
    df <- merge(gmst, era5, by = "row.names")
    row.names(df) = df$Row.names
    
    # fit model to each grid cell (takes about a minute per file, on average)
    system.time({
        mle_gev = lapply(colnames(df)[-(1:2)], function(cnm) {
            
            if(sum(is.na(df[,cnm])) > 2) {
                return(NULL)
            } else {
                df_c = setNames(df[,c("gmst",cnm)], c("gmst", "tmax"))
                df_c = df_c[!is.na(df_c$tmax),]
                return(suppressWarnings(fevd(x=tmax, data=df_c, location.fun=~gmst, type="GEV", optim.args = list(method = "Nelder-Mead"))))
            }
        })
        save(mle_gev, file = paste0("fitted/", gsub("csv", "", gsub("flattened", "shift-ml-ests", fnm)),"Rdata"))
    })["elapsed"]
})

#### **Get pars, return levels**

In [2]:
invisible(sapply(list.files("fitted", pattern = ".Rdata", full.names = T), function(fnm) {
    load(fnm)
    
    # parameter estimates, event value, return period
    ml_res <- data.frame(t(do.call("rbind", lapply(1:length(mle_gev), function(i) {
        if(is.null(mle_gev[[i]])) {
            return(rep(NA,10))
        } else {
            mdl <- mle_gev[[i]]
            
            pars <- mdl$results$par
            
            gmst_1950 <- mdl$cov.data["1950", "gmst"]
            gmst_1980 <- mdl$cov.data["1980", "gmst"]
            gmst_2022 <- mdl$cov.data["2022", "gmst"]
            
            rl20 <- setNames(unname(sapply(c(gmst_2022 - 1.2, gmst_1950, gmst_1980, gmst_2022, gmst_2022 + 0.8), return_level, mdl = mdl, rp = 20)),
                             c("pi", "1950", "1980", "2022", "fut"))
            
            nll = mdl$results$value
            
            return(c(pars, rl20, "nll" = nll))
        }
    }))))
    write.csv(ml_res, gsub("Rdata", "csv", fnm))
}))